In [2]:
# 載入所需要的套件

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
pd.set_option('display.max_rows', None)
from plotly.subplots import make_subplots
import seaborn as sns
import datetime

In [3]:
# 利用 PANDAS 讀入DATA

data = pd.read_csv('covid_19_data.csv')

In [4]:
# 缺失值處理
#Percentage of NAN Values, 計算缺失值在資料內的比例
NAN = [(c, data[c].isna().mean()*100) for c in data]
NAN = pd.DataFrame(NAN, columns=["column_name", "percentage"])
NAN

,column_name,percentage
0,SNo,0.000000
1,ObservationDate,0.000000
2,Province/State,28.352699
3,Country/Region,0.000000
4,Last Update,0.000000
5,Confirmed,0.000000
6,Deaths,0.000000
7,Recovered,0.000000


In [5]:
# fill na of state to unknown
data["Province/State"]= data["Province/State"].fillna('Unknown')

In [8]:
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156292 entries, 0 to 156291
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   SNo              156292 non-null  int64 
 1   ObservationDate  156292 non-null  object
 2   Province/State   156292 non-null  object
 3   Country/Region   156292 non-null  object
 4   Last Update      156292 non-null  object
 5   Confirmed        156292 non-null  int64 
 6   Deaths           156292 non-null  int64 
 7   Recovered        156292 non-null  int64 
dtypes: int64(4), object(4)
memory usage: 9.5+ MB


,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,1,01/22/2020,Anhui,Mainland China,1/22/2020 17:00,1,0,0
1,2,01/22/2020,Beijing,Mainland China,1/22/2020 17:00,14,0,0
2,3,01/22/2020,Chongqing,Mainland China,1/22/2020 17:00,6,0,0
3,4,01/22/2020,Fujian,Mainland China,1/22/2020 17:00,1,0,0
4,5,01/22/2020,Gansu,Mainland China,1/22/2020 17:00,0,0,0


In [10]:
# encoding Confirmed, Deaths, Recovered to int
data[["Confirmed","Deaths","Recovered"]] =data[["Confirmed","Deaths","Recovered"]].astype(int)
# replace Mainland China to China
data['Country/Region'] = data['Country/Region'].replace('Mainland China', 'China')
# ActiveCase = Confirmed - Deaths - Recovered
data['ActiveCase'] = data['Confirmed'] - data['Deaths'] - data['Recovered']
#update index 
Data = data[data['ObservationDate'] == max(data['ObservationDate'])].reset_index()

In [38]:
# cases by contry
Data_country = Data.groupby(["Country/Region"])["Confirmed","ActiveCase","Recovered","Deaths"].sum().reset_index().sort_values("Confirmed",ascending=False).reset_index(drop=True)
# 
# 
fig = go.Figure(data=[go.Bar( name = 'Confirmed',
            x=Data_country['Confirmed'],
            y=Data_country['Country/Region'],
            orientation='h'),
                      go.Bar( name = 'Recovered',
            x=Data_country['Recovered'],
            y=Data_country['Country/Region'],
            orientation='h')
                     ])
fig.update_layout(
    title='Confirmed Cases In Each Country',
    template='plotly_dark',
     xaxis_title="Confirmed Cases",
    yaxis_title="Countries",
)
fig.show()

<ipython-input-38-6248b71161c6>:2: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [19]:
# Confirmed cases
fig = px.choropleth(Data_country, locations=Data_country['Country/Region'],
                    color=Data_country['Confirmed'],locationmode='country names', 
                    hover_name=Data_country['Country/Region'], 
                    color_continuous_scale=px.colors.sequential.Tealgrn,template='plotly_white', )
fig.update_layout(
    title='Confirmed Cases In Each Country',
)
fig.show()


In [20]:
# Recovered cases
fig = px.choropleth(Data_country, locations=Data_country['Country/Region'],
                    color=Data_country['Recovered'],locationmode='country names', 
                    hover_name=Data_country['Country/Region'], 
                    color_continuous_scale=px.colors.sequential.Tealgrn,template='plotly_white', )
fig.update_layout(
    title='Recovered Cases In Each Country',
)
fig.show()